In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/College/vision_research/code/rosetta_analysis/rosetta_neurons

/content/drive/MyDrive/College/vision_research/code/rosetta_analysis/rosetta_neurons


In [6]:
#@title Requirements
!pip install -r requirements.txt
!pip install transformers
!pip install openai-clip
!pip install einops
!pip install pytorch_pretrained_biggan
!pip install Ninja
!pip install timm==0.4.12
!pip install dill

  Cloning https://github.com/openai/CLIP.git (to revision b46f5ac7587d2e1862f8b7b1573179d80dcdd620) to /tmp/pip-install-_kq_sa4w/clip_07fd1f0447bc4e8b9a98c27e8db360d8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-_kq_sa4w/clip_07fd1f0447bc4e8b9a98c27e8db360d8
  Running command git rev-parse -q --verify 'sha^b46f5ac7587d2e1862f8b7b1573179d80dcdd620'
  Running command git fetch -q https://github.com/openai/CLIP.git b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Running command git checkout -q b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/arogozhnikov/einops.git (to revision d6f7910dc80c8413b7bb400f6f6ad24e3c5f5b0f) to /tmp/pip-install-_kq_sa4w/einops_7be518e8d0754fcaa3616fe7f05b6409
  Running command git clone --filter=blob:none --quiet https://github.com/arogozhnikov/eino

In [7]:
#@title Imports
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import os
import math
import match_utils
from match_utils import matching, stats, proggan, nethook, dataset, models, layers, loading, visualize_pairwisematch
device = torch.device('cuda')

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [8]:
# #@title You can use an example table for class 226 used in the paper
# !mkdir "/content/drive/MyDrive/matches/results_226_dino_styleganxl"
# !gdown "https://drive.google.com/uc?export=download&id=1Q9xx-6lEye_MAvVHJkoBXqypKPcIx_fG" -O "/content/drive/MyDrive/matches/results_226_dino_styleganxl/table.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1x3YPe-v9pZtY3YUoYxcdDhlSz-q5nvGJ" -O "/content/drive/MyDrive/matches/results_226_dino_styleganxl/discr_stats.pkl"
# !gdown "https://drive.google.com/uc?export=download&id=1ivJNZmBZuuMCAuPT0s22pLoPdtBx6q9t" -O "/content/drive/MyDrive/matches/results_226_dino_styleganxl/gan_stats.pkl"


In [ ]:
# #@title You can use an example table for class 226 used in the paper
# BASE_DIR="/content/drive/MyDrive/College/vision_research/code/rosetta_analysis/matches/results_226_dino_styleganxl"

# !mkdir -p {BASE_DIR}
# !gdown "https://drive.google.com/uc?export=download&id=1Q9xx-6lEye_MAvVHJkoBXqypKPcIx_fG"   -O {BASE_DIR}/table.pkl
# !gdown "https://drive.google.com/uc?export=download&id=1x3YPe-v9pZtY3YUoYxcdDhlSz-q5nvGJ"   -O {BASE_DIR}/discr_stats.pkl
# !gdown "https://drive.google.com/uc?export=download&id=1ivJNZmBZuuMCAuPT0s22pLoPdtBx6q9t"   -O {BASE_DIR}/gan_stats.pkl


In [38]:
#@title Select Parameters
gan_mode = 'stylegan3' #@param ['biggan','styleganxl','stylegan2-lsun_cat', 'stylegan2-lsun_horse', 'stylegan2-lsun_car','stylegan3', 'projgan']
discr_mode = 'mocov2' #@param ['mae', 'dino_vits16', 'dino_vits8', 'dino_vitb16', 'dino_vitb8', 'dino_resnet50', "mocov2", 'clip', 'resnet50']

if "stylegan2" in gan_mode or gan_mode == "stylegan3" or gan_mode == "projgan":
  classidx  = None
else:
  classidx  = 226 #@param {type:"raw"}

if gan_mode == "stylegan3-afhq":
  !gdown "https://drive.google.com/uc?export=download&id=1TWqhCvcAh2Qcu8XQMDoCKwGkqfOFpqL2"
elif gan_mode == "stylegan2-lsun_car":
  !gdown "https://drive.google.com/uc?export=download&id=1kEZG4_-hFVwEYyYwfmXGbJFu9KFSN53s"
elif gan_mode == "stylegan2-lsun_cat":
  !gdown "https://drive.google.com/uc?export=download&id=1nB5RQPrWRfcLI7yHhKKkf4gjxrJW7HCj"
elif gan_mode == "stylegan2-lsun_horse":
  !gdown "https://drive.google.com/uc?export=download&id=16SHYO836iF1AGgMps-EFXP0TxDaXYp3g"

if discr_mode == "mae":
  !gdown "https://drive.google.com/uc?export=download&id=1uJJHiQ6v8Kec9lpG0RXiyeszfGEMOW-6"

matching_samples = "1600" #@param {type: "string"}

table_path = f"College/vision_research/code/rosetta_analysis/matches/{gan_mode}_{mocov2}_cls{classidx}_{sample_size}"
table_path = "/content/drive/MyDrive/"+table_path
print("Loading from ", table_path)

In [39]:
import importlib
importlib.reload(models)

<module 'match_utils.models' from '/content/drive/MyDrive/College/vision_research/code/rosetta_analysis/rosetta_neurons/match_utils/models.py'>

Loading custom MoCo v2 weights from /content/drive/MyDrive/College/vision_research/code/pretrained_models/moco_v2_shaders21k.pth...


In [16]:
###############################################################
discr, discr_layers = models.load_discr(discr_mode, device)
gan, gan_layers = models.load_gan(gan_mode, device)
ganlayers, discrlayers = layers.get_layers(gan,gan_layers, discr, discr_layers,gan_mode, discr_mode, device)
z_dataset, c_dataset = dataset.create_dataset(gan, gan_mode, 1, 1, classidx, device)

Fall back to downloading from torch hub


Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 166MB/s]


In [17]:
#Load Tables
table, gan_stats, discr_stats = loading.load_stats(table_path, device)
match_scores,_ = torch.max(table,1) # max correlation for each GAN
gan_matches = torch.argmax(table,1) # best D unit for each G unit (for each row)
discr_matches = torch.argmax(table,0) # best G unit for each D unit (for each col)

Loading...
Done


In [18]:
#### Best Buddies
perfect_matches = [] # matched G unit indices
perfect_match_scores = [] # matched pairs correlation
discr_perfect_matches = [] # corresponding matched D unit indices
num_perfect_matches = 0
for i in range(table.shape[0]):
    gan_match = gan_matches[i].item()
    discr_match = discr_matches[gan_match].item()
    if discr_match == i:
        num_perfect_matches+=1
        perfect_matches.append(i)
        discr_perfect_matches.append(gan_match)
        perfect_match_scores.append(match_scores[i])

#sort
gan_match_units = [match for _,match in sorted(zip(perfect_match_scores, perfect_matches), reverse = True)] # matched G units sorted descending by correlation with its D unit
scores = sorted(perfect_match_scores, reverse = True)
scores = [score.item() for score in scores]

#visualize
z_dataset, c_dataset = dataset.create_dataset(gan, gan_mode, 1, 1, classidx, device)
visualize_pairwisematch.viz_matches(table, gan, discr, (z_dataset,c_dataset), ganlayers, discrlayers, gan_stats, discr_stats, gan_mode, discr_mode, gan_match_units, scores )



Output hidden; open in https://colab.research.google.com to view.